In [1]:
import pandas as pd
import numpy as np
import requests
import time
import re

In [2]:
#Importamos el archivo de excel
data_cristiano = pd.read_excel('data_cr70.xlsx')

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_cristiano.dtypes)
data_cristiano

Temporada             object
Competicion           object
Partidos_jugados      object
Goles_anotados        object
Pases_gol             object
Tarjetas_amarillas    object
segundas_amarillas    object
Tarjetas_rojas        object
minutos_jugados       object
html_club             object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,segundas_amarillas,Tarjetas_rojas,minutos_jugados,html_club
0,23/24,Saudi Pro League,15,16,8,-,-,-,1.346',"<img src=""https://tmssl.akamaized.net/images/w..."
1,22/23,Saudi Pro League,16,14,2,3,-,-,1.434',"<img src=""https://tmssl.akamaized.net/images/w..."
2,22/23,Premier League,10,1,-,2,-,-,520',"<img src=""https://tmssl.akamaized.net/images/w..."
3,21/22,Premier League,30,18,3,8,-,-,2.456',"<img src=""https://tmssl.akamaized.net/images/w..."
4,21/22,Serie A,1,-,-,1,-,-,30',"<img src=""https://tmssl.akamaized.net/images/w..."
...,...,...,...,...,...,...,...,...,...,...
85,04/05,Champions League,7,-,2,-,-,-,502',"<img src=""https://tmssl.akamaized.net/images/w..."
86,04/05,Clasificación Champions League,1,-,-,-,-,-,64',"<img src=""https://tmssl.akamaized.net/images/w..."
87,03/04,Champions League,5,-,1,-,-,-,291',"<img src=""https://tmssl.akamaized.net/images/w..."
88,02/03,Copa UEFA,2,-,1,-,-,-,118',"<img src=""https://tmssl.akamaized.net/images/w..."


In [5]:
# Reemplazar guiones '-' en todo el DataFrame por NaN
data_cristiano = data_cristiano.replace('-', np.nan)

# Convertir 'minutos_jugados' a cadena y eliminar apóstrofes y puntos
data_cristiano['minutos_jugados'] = data_cristiano['minutos_jugados'].astype(str)
data_cristiano['minutos_jugados'] = data_cristiano['minutos_jugados'].str.replace("'", '')
data_cristiano['minutos_jugados'] = data_cristiano['minutos_jugados'].str.replace(".", '')

# Convertir la columna 'minutos_jugados' a tipo numérico (float) para manejar NaN
data_cristiano['minutos_jugados'] = pd.to_numeric(data_cristiano['minutos_jugados'], errors='coerce')

# Luego, rellenar los NaN con un valor predeterminado (por ejemplo, 0)
data_cristiano['minutos_jugados'].fillna(0, inplace=True)

# Convertir la columna 'minutos_jugados' a tipo entero
data_cristiano['minutos_jugados'] = data_cristiano['minutos_jugados'].astype(int)

# Visualizar el DataFrame después de la limpieza
print(data_cristiano.dtypes)
data_cristiano

Temporada             object
Competicion           object
Partidos_jugados      object
Goles_anotados        object
Pases_gol             object
Tarjetas_amarillas    object
segundas_amarillas    object
Tarjetas_rojas        object
minutos_jugados        int32
html_club             object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,segundas_amarillas,Tarjetas_rojas,minutos_jugados,html_club
0,23/24,Saudi Pro League,15,16,8,NaN,NaN,NaN,1346,"<img src=""https://tmssl.akamaized.net/images/w..."
1,22/23,Saudi Pro League,16,14,2,3,NaN,NaN,1434,"<img src=""https://tmssl.akamaized.net/images/w..."
2,22/23,Premier League,10,1,NaN,2,NaN,NaN,520,"<img src=""https://tmssl.akamaized.net/images/w..."
3,21/22,Premier League,30,18,3,8,NaN,NaN,2456,"<img src=""https://tmssl.akamaized.net/images/w..."
4,21/22,Serie A,1,NaN,NaN,1,NaN,NaN,30,"<img src=""https://tmssl.akamaized.net/images/w..."
...,...,...,...,...,...,...,...,...,...,...
85,04/05,Champions League,7,NaN,2,NaN,NaN,NaN,502,"<img src=""https://tmssl.akamaized.net/images/w..."
86,04/05,Clasificación Champions League,1,NaN,NaN,NaN,NaN,NaN,64,"<img src=""https://tmssl.akamaized.net/images/w..."
87,03/04,Champions League,5,NaN,1,NaN,NaN,NaN,291,"<img src=""https://tmssl.akamaized.net/images/w..."
88,02/03,Copa UEFA,2,NaN,1,NaN,NaN,NaN,118,"<img src=""https://tmssl.akamaized.net/images/w..."


In [6]:
#Eliminaremos espacios innecesarios en el dataframe
data_cristiano = data_cristiano.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Lista de columnas a convertir a valores numéricos
columnas_numericas = ['Partidos_jugados', 'Goles_anotados', 'Pases_gol', 'Tarjetas_amarillas', 'segundas_amarillas', 'Tarjetas_rojas']

# Convertir las columnas a valores numéricos
data_cristiano[columnas_numericas] = data_cristiano[columnas_numericas].apply(pd.to_numeric, errors='coerce')

# Redondear los valores a enteros después de manejar NaN o inf
data_cristiano[columnas_numericas] = data_cristiano[columnas_numericas].fillna(0).astype(np.float64).round(0).astype(np.int64)


# Visualizar el DataFrame después de la limpieza
print(data_cristiano.dtypes)
data_cristiano

Temporada             object
Competicion           object
Partidos_jugados       int64
Goles_anotados         int64
Pases_gol              int64
Tarjetas_amarillas     int64
segundas_amarillas     int64
Tarjetas_rojas         int64
minutos_jugados        int32
html_club             object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,segundas_amarillas,Tarjetas_rojas,minutos_jugados,html_club
0,23/24,Saudi Pro League,15,16,8,0,0,0,1346,"<img src=""https://tmssl.akamaized.net/images/w..."
1,22/23,Saudi Pro League,16,14,2,3,0,0,1434,"<img src=""https://tmssl.akamaized.net/images/w..."
2,22/23,Premier League,10,1,0,2,0,0,520,"<img src=""https://tmssl.akamaized.net/images/w..."
3,21/22,Premier League,30,18,3,8,0,0,2456,"<img src=""https://tmssl.akamaized.net/images/w..."
4,21/22,Serie A,1,0,0,1,0,0,30,"<img src=""https://tmssl.akamaized.net/images/w..."
...,...,...,...,...,...,...,...,...,...,...
85,04/05,Champions League,7,0,2,0,0,0,502,"<img src=""https://tmssl.akamaized.net/images/w..."
86,04/05,Clasificación Champions League,1,0,0,0,0,0,64,"<img src=""https://tmssl.akamaized.net/images/w..."
87,03/04,Champions League,5,0,1,0,0,0,291,"<img src=""https://tmssl.akamaized.net/images/w..."
88,02/03,Copa UEFA,2,0,1,0,0,0,118,"<img src=""https://tmssl.akamaized.net/images/w..."


In [3]:
# Función para extraer el texto de cada valor en la columna html_club
def extraer_texto(html):
    inicio_texto = html.find('alt="') + len('alt="')
    fin_texto = html.find('"', inicio_texto)
    texto_extraido = html[inicio_texto:fin_texto]
    return texto_extraido

# Aplicar la función a la columna html_club y crear una nueva columna con el texto extraído
data_cristiano['Club'] = data_cristiano['html_club'].apply(extraer_texto)

#Por último procedemos a eliminar la columna del outerhtml que ya no hace falta.
data_cristiano.drop('html_club', axis=1, inplace=True)

print(data_cristiano.dtypes)
data_cristiano

Temporada             object
Competicion           object
Partidos_jugados      object
Goles_anotados        object
Pases_gol             object
Tarjetas_amarillas    object
segundas_amarillas    object
Tarjetas_rojas        object
minutos_jugados       object
Club                  object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,segundas_amarillas,Tarjetas_rojas,minutos_jugados,Club
0,23/24,Saudi Pro League,15,16,8,-,-,-,1.346',Al-Nassr FC
1,22/23,Saudi Pro League,16,14,2,3,-,-,1.434',Al-Nassr FC
2,22/23,Premier League,10,1,-,2,-,-,520',Manchester United
3,21/22,Premier League,30,18,3,8,-,-,2.456',Manchester United
4,21/22,Serie A,1,-,-,1,-,-,30',Juventus de Turín
...,...,...,...,...,...,...,...,...,...,...
85,04/05,Champions League,7,-,2,-,-,-,502',Manchester United
86,04/05,Clasificación Champions League,1,-,-,-,-,-,64',Manchester United
87,03/04,Champions League,5,-,1,-,-,-,291',Manchester United
88,02/03,Copa UEFA,2,-,1,-,-,-,118',Sporting de Lisboa


In [8]:
# Añadiremos una columna con el nombre de cada jugador para luego concatenar todos los dataframes
data_cristiano['Jugador'] = 'Cristiano Ronaldo'

data_cristiano

,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,segundas_amarillas,Tarjetas_rojas,minutos_jugados,Club,Jugador
0,23/24,Saudi Pro League,15,16,8,0,0,0,1346,Al-Nassr FC,Cristiano Ronaldo
1,22/23,Saudi Pro League,16,14,2,3,0,0,1434,Al-Nassr FC,Cristiano Ronaldo
2,22/23,Premier League,10,1,0,2,0,0,520,Manchester United,Cristiano Ronaldo
3,21/22,Premier League,30,18,3,8,0,0,2456,Manchester United,Cristiano Ronaldo
4,21/22,Serie A,1,0,0,1,0,0,30,Juventus de Turín,Cristiano Ronaldo
...,...,...,...,...,...,...,...,...,...,...,...
85,04/05,Champions League,7,0,2,0,0,0,502,Manchester United,Cristiano Ronaldo
86,04/05,Clasificación Champions League,1,0,0,0,0,0,64,Manchester United,Cristiano Ronaldo
87,03/04,Champions League,5,0,1,0,0,0,291,Manchester United,Cristiano Ronaldo
88,02/03,Copa UEFA,2,0,1,0,0,0,118,Sporting de Lisboa,Cristiano Ronaldo


In [9]:
#Procedemos a guardar el dataframe en un archivo xlsx nuevo
nombre_archivo = 'Data Cristiano historico.xlsx'

data_cristiano.to_excel(nombre_archivo, index=False)  # El argumento index=False evita que se incluyan los índices en el archivo

print(f'DataFrame guardado en {nombre_archivo}')

DataFrame guardado en Data Cristiano historico.xlsx


In [26]:
#VAMOS A PROCESAR TODOS LOS PASOS ANTERIORES EN FUNCIONES. 

#Funcion para poder convertir los valores de minutos a numero
def minjugados_dataframe(df):
    # Reemplazar guiones '-' por NaN
    df = df.replace('-', np.nan)

    # Convertir 'minutos_jugados' a cadena y eliminar apóstrofes y puntos
    df['minutos_jugados'] = df['minutos_jugados'].astype(str)
    df['minutos_jugados'] = df['minutos_jugados'].str.replace("'", '')
    df['minutos_jugados'] = df['minutos_jugados'].str.replace(".", '')

    # Convertir la columna 'minutos_jugados' a tipo numérico (float) para manejar NaN
    df['minutos_jugados'] = pd.to_numeric(df['minutos_jugados'], errors='coerce')

    # Luego, rellenar los NaN con un valor predeterminado (por ejemplo, 0)
    df['minutos_jugados'].fillna(0, inplace=True)

    # Convertir la columna 'minutos_jugados' a tipo entero
    df['minutos_jugados'] = df['minutos_jugados'].astype(int)

    return df

#Funcion de limpieza final de datos
def limpiar_y_convertir_numericos(df):
    # Eliminar espacios innecesarios
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    # Lista de columnas a convertir a valores numéricos
    columnas_numericas = ['Partidos_jugados', 'Goles_anotados', 'Pases_gol', 'Tarjetas_amarillas', 'segundas_amarillas', 'Tarjetas_rojas']

    # Convertir las columnas a valores numéricos
    df[columnas_numericas] = df[columnas_numericas].apply(pd.to_numeric, errors='coerce')

    # Redondear los valores a enteros después de manejar NaN o inf
    df[columnas_numericas] = df[columnas_numericas].fillna(0).astype(np.float64).round(0).astype(np.int64)

    return df

In [25]:
#Data de Lionel Messi
data_messi = pd.read_excel(r'C:\Users\Usuario\Documents\PYTHON\Data_futbolistas\data_messi0.xlsx')
data_messi['Jugador'] = 'Lionel Messi'

#Aplicamos todas las funciones al dataframe:
data_messi = minjugados_dataframe(data_messi)
data_messi = limpiar_y_convertir_numericos(data_messi)
data_messi['Club'] = data_messi['html_club'].apply(extraer_texto)

#Por último procedemos a eliminar la columna del outerhtml que ya no hace falta.
data_messi.drop('html_club', axis=1, inplace=True)

print(data_futbol1.dtypes)
data_messi

Temporada             object
Competicion           object
Partidos_jugados       int64
Goles_anotados         int64
Pases_gol              int64
Tarjetas_amarillas     int64
Tarjetas_rojas         int64
minutos_jugados        int32
Jugador               object
segundas_amarillas     int64
Club                  object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,Tarjetas_rojas,minutos_jugados,Jugador,Club
0,2023,MLS,6,1,2,0,0,372,Lionel Messi,Inter Miami CF
1,22/23,Ligue 1,32,16,16,0,0,2842,Lionel Messi,París Saint-Germain FC
2,21/22,Ligue 1,26,6,15,0,0,2153,Lionel Messi,París Saint-Germain FC
3,20/21,LaLiga,35,30,11,4,0,3022,Lionel Messi,FC Barcelona
4,19/20,LaLiga,33,25,22,4,0,2879,Lionel Messi,FC Barcelona
...,...,...,...,...,...,...,...,...,...,...
77,07/08,Champions League,9,6,2,2,0,726,Lionel Messi,FC Barcelona
78,06/07,Champions League,5,1,0,1,0,385,Lionel Messi,FC Barcelona
79,06/07,Supercopa de Europa,1,0,0,0,0,90,Lionel Messi,FC Barcelona
80,05/06,Champions League,6,1,2,0,0,322,Lionel Messi,FC Barcelona


In [27]:
#Ahora procederemos a importar la data del resto de futbolistas
#Data de Karim Benzema
data_benzema = pd.read_excel(r'C:\Users\Usuario\Documents\PYTHON\Data_futbolistas\data_benze0.xlsx')
data_benzema['Jugador'] = 'Karim Benzema'
#Data de Ryan Giggs
data_giggs = pd.read_excel(r'C:\Users\Usuario\Documents\PYTHON\Data_futbolistas\data_giggs0.xlsx')
data_giggs['Jugador'] = 'Ryan Giggs'
#Data de David Beckham
data_beckham = pd.read_excel(r'C:\Users\Usuario\Documents\PYTHON\Data_futbolistas\data_beck0.xlsx')
data_beckham['Jugador'] = 'David Beckham'
#Data de Angel Di Maria
data_dimar = pd.read_excel(r'C:\Users\Usuario\Documents\PYTHON\Data_futbolistas\data_dima0.xlsx')
data_dimar['Jugador'] = 'Angel Di Maria'

#Procedemos a concatenar todos los dataframes en uno solo.
data_futbol1 = pd.concat([data_benzema, data_giggs, data_beckham, data_dimar])

# Visualizamos el DataFrame y el tipo de datos que contiene
print(data_futbol1.dtypes)
data_futbol1

Temporada             object
Competicion           object
Partidos_jugados      object
Goles_anotados        object
Pases_gol             object
Tarjetas_amarillas    object
minutos_jugados       object
html_club             object
Jugador               object
segundas_amarillas    object
Tarjetas_rojas        object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,minutos_jugados,html_club,Jugador,segundas_amarillas,Tarjetas_rojas
0,23/24,Saudi Pro League,13,9,4,-,1.070',"<a title=""Al-Ittihad Club"" href=""/al-ittihad-d...",Karim Benzema,NaN,NaN
1,22/23,LaLiga,24,19,3,1,2.044',"<a title=""Real Madrid CF"" href=""/real-madrid/s...",Karim Benzema,NaN,NaN
2,21/22,LaLiga,32,27,12,-,2.603',"<a title=""Real Madrid CF"" href=""/real-madrid/s...",Karim Benzema,NaN,NaN
3,20/21,LaLiga,34,23,9,2,2.901',"<a title=""Real Madrid CF"" href=""/real-madrid/s...",Karim Benzema,NaN,NaN
4,19/20,LaLiga,37,21,8,-,3.156',"<a title=""Real Madrid CF"" href=""/real-madrid/s...",Karim Benzema,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
70,08/09,Copa UEFA,5,1,-,2,341',"<img src=""https://tmssl.akamaized.net/images/w...",Angel Di Maria,-,-
71,07/08,Copa UEFA,4,1,-,-,133',"<img src=""https://tmssl.akamaized.net/images/w...",Angel Di Maria,-,-
72,07/08,Champions League,6,-,1,1,292',"<img src=""https://tmssl.akamaized.net/images/w...",Angel Di Maria,-,-
73,07/08,Clasificación Champions League,1,-,-,-,73',"<img src=""https://tmssl.akamaized.net/images/w...",Angel Di Maria,-,-


In [28]:
#Aplicamos todas las funciones al dataframe:
data_futbol1 = minjugados_dataframe(data_futbol1)
data_futbol1 = limpiar_y_convertir_numericos(data_futbol1)
data_futbol1['Club'] = data_futbol1['html_club'].apply(extraer_texto)

#Por último procedemos a eliminar la columna del outerhtml que ya no hace falta.
data_futbol1.drop('html_club', axis=1, inplace=True)

print(data_futbol1.dtypes)
data_futbol1

Temporada             object
Competicion           object
Partidos_jugados       int64
Goles_anotados         int64
Pases_gol              int64
Tarjetas_amarillas     int64
minutos_jugados        int32
Jugador               object
segundas_amarillas     int64
Tarjetas_rojas         int64
Club                  object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,minutos_jugados,Jugador,segundas_amarillas,Tarjetas_rojas,Club
0,23/24,Saudi Pro League,13,9,4,0,1070,Karim Benzema,0,0,Al-Ittihad Club
1,22/23,LaLiga,24,19,3,1,2044,Karim Benzema,0,0,Real Madrid CF
2,21/22,LaLiga,32,27,12,0,2603,Karim Benzema,0,0,Real Madrid CF
3,20/21,LaLiga,34,23,9,2,2901,Karim Benzema,0,0,Real Madrid CF
4,19/20,LaLiga,37,21,8,0,3156,Karim Benzema,0,0,Real Madrid CF
...,...,...,...,...,...,...,...,...,...,...,...
70,08/09,Copa UEFA,5,1,0,2,341,Angel Di Maria,0,0,SL Benfica
71,07/08,Copa UEFA,4,1,0,0,133,Angel Di Maria,0,0,SL Benfica
72,07/08,Champions League,6,0,1,1,292,Angel Di Maria,0,0,SL Benfica
73,07/08,Clasificación Champions League,1,0,0,0,73,Angel Di Maria,0,0,SL Benfica


In [29]:
#Uniremos el dataframe creado con la data de Cristiano que usamos como dataframe inicial para probar toda la limpieza.
data_futbol = pd.concat([data_cristiano, data_messi, data_futbol1])

#Visualizamos el dataframe
print(data_futbol.dtypes)
data_futbol

Temporada              object
Competicion            object
Partidos_jugados        int64
Goles_anotados          int64
Pases_gol               int64
Tarjetas_amarillas      int64
segundas_amarillas    float64
Tarjetas_rojas          int64
minutos_jugados         int32
Club                   object
Jugador                object
dtype: object


,Temporada,Competicion,Partidos_jugados,Goles_anotados,Pases_gol,Tarjetas_amarillas,segundas_amarillas,Tarjetas_rojas,minutos_jugados,Club,Jugador
0,23/24,Saudi Pro League,15,16,8,0,0.0,0,1346,Al-Nassr FC,Cristiano Ronaldo
1,22/23,Saudi Pro League,16,14,2,3,0.0,0,1434,Al-Nassr FC,Cristiano Ronaldo
2,22/23,Premier League,10,1,0,2,0.0,0,520,Manchester United,Cristiano Ronaldo
3,21/22,Premier League,30,18,3,8,0.0,0,2456,Manchester United,Cristiano Ronaldo
4,21/22,Serie A,1,0,0,1,0.0,0,30,Juventus de Turín,Cristiano Ronaldo
...,...,...,...,...,...,...,...,...,...,...,...
70,08/09,Copa UEFA,5,1,0,2,0.0,0,341,SL Benfica,Angel Di Maria
71,07/08,Copa UEFA,4,1,0,0,0.0,0,133,SL Benfica,Angel Di Maria
72,07/08,Champions League,6,0,1,1,0.0,0,292,SL Benfica,Angel Di Maria
73,07/08,Clasificación Champions League,1,0,0,0,0.0,0,73,SL Benfica,Angel Di Maria


In [30]:
#Procedemos a guardar el dataframe en un archivo xlsx nuevo
nombre_archivo = 'Data futbolistas historico.xlsx'

data_futbol.to_excel(nombre_archivo, index=False)  # El argumento index=False evita que se incluyan los índices en el archivo

print(f'DataFrame guardado en {nombre_archivo}')

DataFrame guardado en Data futbolistas historico.xlsx


Ahora, extraeremos data los partidos de la champions league de Cristiano Ronaldo y Messi para hacer la comparación.

In [4]:
#Importamos el archivo de excel
data_cristianochampions = pd.read_excel('Ronaldo_champions.xlsx')
data_cristianochampions['Jugador'] = 'Cristiano Ronaldo'
data_messichamphions = pd.read_excel('Messi_champions.xlsx')
data_messichamphions['Jugador'] = 'Lionel Messi'

#Procedemos a concatenar todos los dataframes en uno solo.
data_champions = pd.concat([data_cristianochampions, data_messichamphions])

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_champions.dtypes)
data_champions

Fase_Competicion     object
Fecha                object
Localidad            object
Rival                object
Resultado            object
Posicion             object
Goles_anotados      float64
Asistencias         float64
Minutos_jugados      object
Link_club            object
Jugador              object
dtype: object


,Fase_Competicion,Fecha,Localidad,Rival,Resultado,Posicion,Goles_anotados,Asistencias,Minutos_jugados,Link_club,Jugador
0,Grupo E,16/09/03,H,Panathinaikos,5:0,Suplente,NaN,NaN,NaN,"<img src=""https://tmssl.akamaized.net/images/w...",Cristiano Ronaldo
1,Grupo E,01/10/03,A,VfB Stuttgart,2:1,EI,NaN,1.0,90',"<img src=""https://tmssl.akamaized.net/images/w...",Cristiano Ronaldo
2,Grupo E,22/10/03,A,Rangers FC,0:1,Suplente,NaN,NaN,NaN,"<img src=""https://tmssl.akamaized.net/images/w...",Cristiano Ronaldo
3,Grupo E,04/11/03,H,Rangers FC,3:0,ED,NaN,NaN,90',"<img src=""https://tmssl.akamaized.net/images/w...",Cristiano Ronaldo
4,Grupo E,26/11/03,A,Panathinaikos,0:1,ED,NaN,NaN,90',"<img src=""https://tmssl.akamaized.net/images/w...",Cristiano Ronaldo
...,...,...,...,...,...,...,...,...,...,...,...
196,Grupo H,11/10/22,H,Benfica,1:1,Problemas de pantorrillas,NaN,NaN,NaN,"<img src=""https://tmssl.akamaized.net/images/w...",Lionel Messi
197,Grupo H,25/10/22,H,Maccabi Haifa,7:2,MP,2.0,2.0,90',"<img src=""https://tmssl.akamaized.net/images/w...",Lionel Messi
198,Grupo H,02/11/22,A,Juventus,1:2,MP,NaN,1.0,90',"<img src=""https://tmssl.akamaized.net/images/w...",Lionel Messi
199,Octavos - ida,14/02/23,H,Bayern Múnich,0:1,DEL,NaN,NaN,90',"<img src=""https://tmssl.akamaized.net/images/w...",Lionel Messi


In [5]:
# Aplicaremos la función a la columna link_club y crear una nueva columna con el texto extraído
data_champions['Club'] = data_champions['Link_club'].apply(extraer_texto)

#Por último procedemos a eliminar la columna del outerhtml que ya no hace falta.
data_champions.drop('Link_club', axis=1, inplace=True)

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_champions.dtypes)
data_champions

Fase_Competicion     object
Fecha                object
Localidad            object
Rival                object
Resultado            object
Posicion             object
Goles_anotados      float64
Asistencias         float64
Minutos_jugados      object
Jugador              object
Club                 object
dtype: object


,Fase_Competicion,Fecha,Localidad,Rival,Resultado,Posicion,Goles_anotados,Asistencias,Minutos_jugados,Jugador,Club
0,Grupo E,16/09/03,H,Panathinaikos,5:0,Suplente,NaN,NaN,NaN,Cristiano Ronaldo,Manchester United
1,Grupo E,01/10/03,A,VfB Stuttgart,2:1,EI,NaN,1.0,90',Cristiano Ronaldo,Manchester United
2,Grupo E,22/10/03,A,Rangers FC,0:1,Suplente,NaN,NaN,NaN,Cristiano Ronaldo,Manchester United
3,Grupo E,04/11/03,H,Rangers FC,3:0,ED,NaN,NaN,90',Cristiano Ronaldo,Manchester United
4,Grupo E,26/11/03,A,Panathinaikos,0:1,ED,NaN,NaN,90',Cristiano Ronaldo,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...
196,Grupo H,11/10/22,H,Benfica,1:1,Problemas de pantorrillas,NaN,NaN,NaN,Lionel Messi,París Saint-Germain FC
197,Grupo H,25/10/22,H,Maccabi Haifa,7:2,MP,2.0,2.0,90',Lionel Messi,París Saint-Germain FC
198,Grupo H,02/11/22,A,Juventus,1:2,MP,NaN,1.0,90',Lionel Messi,París Saint-Germain FC
199,Octavos - ida,14/02/23,H,Bayern Múnich,0:1,DEL,NaN,NaN,90',Lionel Messi,París Saint-Germain FC


In [6]:
#Empezaremos la limpieza modificando el tipo de dato en cada columna
# Convertiremos los 'minutos_jugados' a cadena y eliminar apóstrofes y puntos
data_champions['Minutos_jugados'] = data_champions['Minutos_jugados'].astype(str)
data_champions['Minutos_jugados'] = data_champions['Minutos_jugados'].str.replace("'", '')

# Convertiremos la columna 'minutos_jugados' a tipo numérico (float) para manejar NaN
data_champions['Minutos_jugados'] = pd.to_numeric(data_champions['Minutos_jugados'], errors='coerce')

# Rellenaremos los NaN con un valor predeterminado (por ejemplo, 0)
data_champions['Minutos_jugados'].fillna(0, inplace=True)

# Convertir la columna 'minutos_jugados' a tipo entero
data_champions['Minutos_jugados'] = data_champions['Minutos_jugados'].astype(int)

# Lista de columnas a convertir a valores numéricos
columnas_numericas = ['Goles_anotados', 'Asistencias']

# Convertir las columnas a valores numéricos
data_champions[columnas_numericas] = data_champions[columnas_numericas].apply(pd.to_numeric, errors='coerce')

# Redondear los valores a enteros después de manejar NaN o inf
data_champions[columnas_numericas] = data_champions[columnas_numericas].fillna(0).astype(np.float64).round(0).astype(np.int64)

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_champions.dtypes)
data_champions

Fase_Competicion    object
Fecha               object
Localidad           object
Rival               object
Resultado           object
Posicion            object
Goles_anotados       int64
Asistencias          int64
Minutos_jugados      int32
Jugador             object
Club                object
dtype: object


,Fase_Competicion,Fecha,Localidad,Rival,Resultado,Posicion,Goles_anotados,Asistencias,Minutos_jugados,Jugador,Club
0,Grupo E,16/09/03,H,Panathinaikos,5:0,Suplente,0,0,0,Cristiano Ronaldo,Manchester United
1,Grupo E,01/10/03,A,VfB Stuttgart,2:1,EI,0,1,90,Cristiano Ronaldo,Manchester United
2,Grupo E,22/10/03,A,Rangers FC,0:1,Suplente,0,0,0,Cristiano Ronaldo,Manchester United
3,Grupo E,04/11/03,H,Rangers FC,3:0,ED,0,0,90,Cristiano Ronaldo,Manchester United
4,Grupo E,26/11/03,A,Panathinaikos,0:1,ED,0,0,90,Cristiano Ronaldo,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...
196,Grupo H,11/10/22,H,Benfica,1:1,Problemas de pantorrillas,0,0,0,Lionel Messi,París Saint-Germain FC
197,Grupo H,25/10/22,H,Maccabi Haifa,7:2,MP,2,2,90,Lionel Messi,París Saint-Germain FC
198,Grupo H,02/11/22,A,Juventus,1:2,MP,0,1,90,Lionel Messi,París Saint-Germain FC
199,Octavos - ida,14/02/23,H,Bayern Múnich,0:1,DEL,0,0,90,Lionel Messi,París Saint-Germain FC


In [7]:
#Por ultimo modificaremos la columna 'Fase_Competicion' para que se divida por: fase de grupos, octavos... hasta la final

# Patrón de búsqueda para encontrar 'Grupo' seguido de cualquier texto
patron_grupo = r'Grupo \w+'

# Reemplazar las coincidencias del patrón con una cadena vacía ''
data_champions['Fase_Competicion'] = data_champions['Fase_Competicion'].str.replace(patron_grupo, 'Fase de Grupos', regex=True)

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_champions.dtypes)
data_champions

Fase_Competicion    object
Fecha               object
Localidad           object
Rival               object
Resultado           object
Posicion            object
Goles_anotados       int64
Asistencias          int64
Minutos_jugados      int32
Jugador             object
Club                object
dtype: object


,Fase_Competicion,Fecha,Localidad,Rival,Resultado,Posicion,Goles_anotados,Asistencias,Minutos_jugados,Jugador,Club
0,Fase de Grupos,16/09/03,H,Panathinaikos,5:0,Suplente,0,0,0,Cristiano Ronaldo,Manchester United
1,Fase de Grupos,01/10/03,A,VfB Stuttgart,2:1,EI,0,1,90,Cristiano Ronaldo,Manchester United
2,Fase de Grupos,22/10/03,A,Rangers FC,0:1,Suplente,0,0,0,Cristiano Ronaldo,Manchester United
3,Fase de Grupos,04/11/03,H,Rangers FC,3:0,ED,0,0,90,Cristiano Ronaldo,Manchester United
4,Fase de Grupos,26/11/03,A,Panathinaikos,0:1,ED,0,0,90,Cristiano Ronaldo,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...
196,Fase de Grupos,11/10/22,H,Benfica,1:1,Problemas de pantorrillas,0,0,0,Lionel Messi,París Saint-Germain FC
197,Fase de Grupos,25/10/22,H,Maccabi Haifa,7:2,MP,2,2,90,Lionel Messi,París Saint-Germain FC
198,Fase de Grupos,02/11/22,A,Juventus,1:2,MP,0,1,90,Lionel Messi,París Saint-Germain FC
199,Octavos - ida,14/02/23,H,Bayern Múnich,0:1,DEL,0,0,90,Lionel Messi,París Saint-Germain FC


In [8]:
#Aplicamos lo mismo para todas las fases
# Patrón de búsqueda para encontrar 'Octavos' seguido de cualquier texto y asi sucesivamente
patron_octavos = r'Octavos.*'
patron_cuartos = r'Cuartos.*'
patron_semis = r'Semifinal.*'

# Reemplazar las coincidencias del patrón con una cadena vacía ''
data_champions['Fase_Competicion'] = data_champions['Fase_Competicion'].str.replace(patron_octavos, 'Octavos de final', regex=True)
data_champions['Fase_Competicion'] = data_champions['Fase_Competicion'].str.replace(patron_cuartos, 'Cuartos de final', regex=True)
data_champions['Fase_Competicion'] = data_champions['Fase_Competicion'].str.replace(patron_semis, 'Semifinales', regex=True)

#Visualizamos el dataframe y el tipo de datos que contiene
categorias = data_champions['Fase_Competicion'].unique()
print(categorias)

['Fase de Grupos' 'Octavos de final' 'Cuartos de final' 'Semifinales'
 'Final' 'Tercera ronda - ida' 'Tercera ronda - vuelta']


In [9]:
#Como observamos aun debemos limpiar las rondas, que corresponden a la fase clasificatoria.
# Patrón de búsqueda para encontrar 'Octavos' seguido de cualquier texto y asi sucesivamente
patron_rondas = r'Tercera ronda.*'

data_champions['Fase_Competicion'] = data_champions['Fase_Competicion'].str.replace(patron_rondas, 'Fase clasificatoria', regex=True)

#Visualizamos el dataframe y el tipo de datos que contiene
categorias = data_champions['Fase_Competicion'].unique()
print(categorias)

['Fase de Grupos' 'Octavos de final' 'Cuartos de final' 'Semifinales'
 'Final' 'Fase clasificatoria']


In [10]:
#Por último reemplazar las cateogorias de localidad por Visitante o local
# Reemplazar las coincidencias del patrón con una cadena vacía ''
data_champions['Localidad'] = data_champions['Localidad'].str.replace('H', 'Local', regex=True)
data_champions['Localidad'] = data_champions['Localidad'].str.replace('A', 'Visitante', regex=True)

#Por ultimo eliminaremos espacios innecesarios en el dataframe
data_champions = data_champions.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#Visualizamos el dataframe y el tipo de datos que contiene
categorias = data_champions['Localidad'].unique()
print(categorias)

['Local' 'Visitante']


In [13]:
#Procedemos a guardar el dataframe en un archivo xlsx nuevo
nombre_archivo = 'Data Champions historico.xlsx'

data_champions.to_excel(nombre_archivo, index=False)  # El argumento index=False evita que se incluyan los índices en el archivo

print(f'DataFrame guardado en {nombre_archivo}')

DataFrame guardado en Data Champions historico.xlsx


In [4]:
#Importamos el archivo de excel
data_cristianoliga = pd.read_excel('Ronaldo_liga.xlsx')
data_cristianoliga['Jugador'] = 'Cristiano Ronaldo'
data_messiliga = pd.read_excel('Messi_liga.xlsx')
data_messiliga['Jugador'] = 'Lionel Messi'

#Procedemos a concatenar todos los dataframes en uno solo.
data_liga = pd.concat([data_cristianoliga, data_messiliga])

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_liga.dtypes)
data_liga

Fecha               object
Localidad           object
Rival               object
Resultado           object
Posicion            object
Goles_anotados     float64
Asistencias        float64
Minutos_jugados     object
Jugador             object
Unnamed: 8         float64
Unnamed: 9         float64
Unnamed: 10        float64
Unnamed: 11         object
dtype: object


,Fecha,Localidad,Rival,Resultado,Posicion,Goles_anotados,Asistencias,Minutos_jugados,Jugador,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,29/08/09,H,RC Deportivo,3:2,EI,1.0,NaN,90',Cristiano Ronaldo,NaN,NaN,NaN,NaN
1,12/09/09,A,RCD Espanyol,0:3,EI,1.0,NaN,25',Cristiano Ronaldo,NaN,NaN,NaN,NaN
2,20/09/09,H,Xerez CD,5:0,ED,2.0,NaN,79',Cristiano Ronaldo,NaN,NaN,NaN,NaN
3,23/09/09,A,Villarreal CF,0:2,DEL,1.0,NaN,90',Cristiano Ronaldo,NaN,NaN,NaN,NaN
4,26/09/09,H,CD Tenerife,3:0,EI,NaN,NaN,78',Cristiano Ronaldo,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,02/05/21,A,Valencia CF,2:3,DEL,2.0,NaN,90',Lionel Messi,NaN,NaN,NaN,NaN
654,08/05/21,H,Atlético Madrid,0:0,MP,NaN,NaN,90',Lionel Messi,NaN,NaN,NaN,NaN
655,11/05/21,A,Levante UD,3:3,MP,1.0,NaN,90',Lionel Messi,NaN,NaN,NaN,NaN
656,16/05/21,H,RC Celta,1:2,MP,1.0,NaN,90',Lionel Messi,NaN,NaN,NaN,NaN


In [5]:
#Empezaremos la limpieza modificando el tipo de dato en cada columna
# Convertiremos los 'minutos_jugados' a cadena y eliminar apóstrofes y puntos
data_liga['Minutos_jugados'] = data_liga['Minutos_jugados'].astype(str)
data_liga['Minutos_jugados'] = data_liga['Minutos_jugados'].str.replace("'", '')

# Convertiremos la columna 'minutos_jugados' a tipo numérico (float) para manejar NaN
data_liga['Minutos_jugados'] = pd.to_numeric(data_liga['Minutos_jugados'], errors='coerce')

# Rellenaremos los NaN con un valor predeterminado (por ejemplo, 0)
data_liga['Minutos_jugados'].fillna(0, inplace=True)

# Convertir la columna 'minutos_jugados' a tipo entero
data_liga['Minutos_jugados'] = data_liga['Minutos_jugados'].astype(int)

# Lista de columnas a convertir a valores numéricos
columnas_numericas = ['Goles_anotados', 'Asistencias']

# Convertir las columnas a valores numéricos
data_liga[columnas_numericas] = data_liga[columnas_numericas].apply(pd.to_numeric, errors='coerce')

# Redondear los valores a enteros después de manejar NaN o inf
data_liga[columnas_numericas] = data_liga[columnas_numericas].fillna(0).astype(np.float64).round(0).astype(np.int64)

#Por último procedemos a eliminar las columnas no definidas del dataframe
columnas_eliminar = ['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']
data_liga.drop(columnas_eliminar, axis=1, inplace=True)

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_liga.dtypes)
data_liga

Fecha              object
Localidad          object
Rival              object
Resultado          object
Posicion           object
Goles_anotados      int64
Asistencias         int64
Minutos_jugados     int32
Jugador            object
dtype: object


,Fecha,Localidad,Rival,Resultado,Posicion,Goles_anotados,Asistencias,Minutos_jugados,Jugador
0,29/08/09,H,RC Deportivo,3:2,EI,1,0,90,Cristiano Ronaldo
1,12/09/09,A,RCD Espanyol,0:3,EI,1,0,25,Cristiano Ronaldo
2,20/09/09,H,Xerez CD,5:0,ED,2,0,79,Cristiano Ronaldo
3,23/09/09,A,Villarreal CF,0:2,DEL,1,0,90,Cristiano Ronaldo
4,26/09/09,H,CD Tenerife,3:0,EI,0,0,78,Cristiano Ronaldo
...,...,...,...,...,...,...,...,...,...
653,02/05/21,A,Valencia CF,2:3,DEL,2,0,90,Lionel Messi
654,08/05/21,H,Atlético Madrid,0:0,MP,0,0,90,Lionel Messi
655,11/05/21,A,Levante UD,3:3,MP,1,0,90,Lionel Messi
656,16/05/21,H,RC Celta,1:2,MP,1,0,90,Lionel Messi


In [6]:
#Procedemos a guardar el dataframe en un archivo xlsx nuevo
nombre_archivo = 'Data Liga historico.xlsx'

data_liga.to_excel(nombre_archivo, index=False)  # El argumento index=False evita que se incluyan los índices en el archivo

print(f'DataFrame guardado en {nombre_archivo}')

DataFrame guardado en Data Liga historico.xlsx
